In [1]:
%matplotlib inline
import geopandas as gpd
import numpy as np
import pandas as pd
import pandas_gbq
import rioxarray
import xarray
from scipy.interpolate import interp2d
from tqdm import tqdm
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

import os

In [2]:
pid = 'spike-sandbox'

# Extraemos la base de latitud/longitud

In [6]:
query = '''
SELECT *
FROM `CR2.emisiones_PM25`
WHERE Time="2015-05-01"
'''
df = pandas_gbq.read_gbq(query, project_id = pid, use_bqstorage_api=True)

Downloading: 100%|██████████| 296792/296792 [00:35<00:00, 8298.55rows/s] 


In [18]:
gdf = gpd.GeoDataFrame(df, geometry = gpd.points_from_xy(df.lon, df.lat))
gdf.crs = {'init':'epsg:4326'}

/Users/pipe/anaconda3/envs/spike_basicoV6/lib/python3.6/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [12]:
gdf.head()

,Time,lat,lon,EMI_PM25,geometry
0,2015-05-01 00:00:00+00:00,-48.40,-72.88,1.886276e-19,POINT (-72.88000 -48.40000)
1,2015-05-01 00:00:00+00:00,-48.22,-72.64,2.148773e-18,POINT (-72.64000 -48.22000)
2,2015-05-01 00:00:00+00:00,-48.18,-72.14,3.140934e-19,POINT (-72.14000 -48.18000)
3,2015-05-01 00:00:00+00:00,-47.88,-73.48,2.076368e-18,POINT (-73.48000 -47.88000)
4,2015-05-01 00:00:00+00:00,-47.76,-73.60,1.655012e-18,POINT (-73.60000 -47.76000)


In [13]:
comunas = gpd.read_file("/Users/pipe/Documents/Spike/CR2/datos/mapas_censo/Comunas/comunas.shp")
comunas.columns = [m.lower() for m in comunas.columns]
comunas = comunas.to_crs({'init': 'epsg:4326'})
comunas.head(2)

/Users/pipe/anaconda3/envs/spike_basicoV6/lib/python3.6/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


,objectid,shape_leng,dis_elec,cir_sena,cod_comuna,codregion,st_area_sh,st_length_,region,comuna,provincia,geometry
0,48,170038.624165,16,8,6204,6,9.685774e+08,206184.271675,Región del Libertador Bernardo O'Higgins,Marchigüe,Cardenal Caro,"POLYGON ((-71.80071 -34.20728, -71.79992 -34.2..."
1,29,125730.104795,15,8,6102,6,4.157446e+08,151911.576827,Región del Libertador Bernardo O'Higgins,Codegua,Cachapoal,"POLYGON ((-70.35293 -33.95263, -70.35243 -33.9..."


### Cruzamos cada geopandas (para cada tiempo) con las regiones y guardamos los archivos


In [23]:
gdf = gpd.sjoin(gdf, comunas[['cod_comuna','codregion', 'region', 'comuna', 'provincia', 'geometry']],
                how='inner', op='within')

In [27]:
pandas_gbq.to_gbq(gdf[['lat', 'lon', 'cod_comuna', 'codregion', 'region', 'comuna', 'provincia']], 
                  'CR2.mapeo_latlon_comuna_region', 
                  project_id=pid, 
                  if_exists='append',
                  chunksize = int(1e6))

1it [00:16, 16.26s/it]
